In [2]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [3]:
def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    text = re.sub(r'<img.+?>(<br ?/?>)?', '', text)
    text = re.sub(r'^<br ?/?>', '', text)
    return text.strip()

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews")
    if url_normal == None:
        url_normal = soup.find('p', id="regularnews")
    url_normal = url_normal.a.get('href')
    if '/http://' in url_normal:
        url_normal = 'http://' + url_normal.split('/http://')[-1]
    else:
        url_normal = 'https://' + url_normal.split('/https://')[-1]
    date = soup.find('p', id="newsDate").text[1:-1]
    #title_easy = soup.find('h1', class_="article-main__title")
    #title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

# for new web easy
def scrape_easy_one_new(html):
    soup = BeautifulSoup(html, "html.parser")
    url_easy = 'https://' + soup.find('meta', property="og:url").get('content').split('/https://')[-1]
    url_normal = soup.find('div', class_="link-to-normal").a.get('href')
    date = soup.find('p', class_="article-main__date").text[1:-1]
    title_easy = soup.find('h1', class_="article-main__title")
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', class_="article-main__body article-body")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', class_="article-main__body article-body").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }



def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    with open('nolinknormal.txt') as f:
        nolink = f.read().split()
    return sorted(set(notyet) - set(nolink))
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

# scrape

In [4]:
driver = webdriver.Chrome()

# clean category & keyword

In [4]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

8398
4744


In [47]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  8730


[('社会', 2617),
 ('国際', 2336),
 ('科学・文化', 1443),
 ('ビジネス', 1345),
 ('スポーツ', 1071),
 ('政治', 772),
 ('暮らし', 732),
 ('地域', 615),
 ('気象・災害', 414)]

In [46]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ":
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [3]:
normal = pd.read_json('nhkwebeasy.json')
normal.id.value_counts()

k10010353911000    1
k10010441541000    1
k10012115201000    1
k10010055061000    1
k10010471031000    1
                  ..
k10010771931000    1
k10010399941000    1
k10010904951000    1
k10011871991000    1
k10011099221000    1
Name: id, Length: 4530, dtype: int64

# NHK web easy

In [4]:
with open('linkeasy.txt') as f:
    urls = f.read().strip().split()
    ids = set([x.split('.html')[0].split('/')[-1] for x in urls])

id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
print(len(ids), len(id_exist))
print('not yet', len(ids - id_exist))
urls[:10]

918 4916
not yet 826


['https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014345501000/k10014345501000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014358721000/k10014358721000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014360391000/k10014360391000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014364701000/k10014364701000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014407931000/k10014407931000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014420291000/k10014420291000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014425011000/k10014425011000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014425961000/k10014425961000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014450891000/k10014450891000.html',
 'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014466

In [4]:
driver = webdriver.Chrome()

In [42]:
id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())
for url in urls:
    ID = url.split('.html')[0].split('/')[-1]
    if ID in id_exist:
        continue
    driver.get(url)
    time.sleep(4)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if snap == None:
        continue
    archiveurl = 'https://web.archive.org' + snap.group(2)
    html = requests.get(archiveurl).text
    time.sleep(3)
    """
    driver.get(archiveurl)
    time.sleep(10)
    html = driver.page_source.encode('utf-8')
    """
    try:
        dic = scrape_easy_one(html)
    except:
        dic = scrape_easy_one_new(html)
    js_e(dic)
    print(archiveurl, end='\n')
    id_exist = set(pd.read_json('nhkwebeasy.json')['id'].tolist())

https://web.archive.org/web/20140220102100/http://www3.nhk.or.jp/news/easy/k10014817011000/k10014817011000.html SUCCESShttps://web.archive.org/web/20150202040756/http://www3.nhk.or.jp/news/easy/k10014850531000/k10014850531000.html SUCCESShttps://web.archive.org/web/20140220114955/http://www3.nhk.or.jp/news/easy/k10014866311000/k10014866311000.html SUCCESShttps://web.archive.org/web/20140220231927/http://www3.nhk.or.jp/news/easy/k10014876541000/k10014876541000.html SUCCESShttps://web.archive.org/web/20150202040847/http://www3.nhk.or.jp/news/easy/k10014903541000/k10014903541000.html SUCCESShttps://web.archive.org/web/20150202040754/http://www3.nhk.or.jp/news/easy/k10014903841000/k10014903841000.html SUCCESShttps://web.archive.org/web/20150128133837/http://www3.nhk.or.jp/news/easy/k10014958811000/k10014958811000.html SUCCESShttps://web.archive.org/web/20150318032042/http://www3.nhk.or.jp/news/easy/k10015570521000/k10015570521000.html SUCCESShttps://web.archive.org/web/20150303031941/http:

AttributeError: 'NoneType' object has no attribute 'get'

In [39]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
dic

{'id': 'k10014838111000',
 'title_easy': 'インドの人工衛星がアジアで初めて火星の軌道に入る',
 'title_easy_ruby': 'インドの<ruby>人工衛星<rt>じんこうえいせい</rt></ruby>がアジアで<ruby>初<rt>はじ</rt></ruby>めて<ruby>火星<rt>かせい</rt></ruby>の<ruby>軌道<rt>きどう</rt></ruby>に<ruby>入<rt>はい</rt></ruby>る',
 'article_easy': '<plc>インド</plc>は去年１１月、火星を調べる人工衛星「マンガルヤーン」を打ち上げました。「マンガルヤーン」は１０か月で約６億５０００万ｋｍ飛んで、火星の近くまで行きました。\n２４日、「マンガルヤーン」はスピードと方向を少しずつ変えながら火星を回る軌道に入りました。<per>モディ</per>首相もこのとき、<org>インド宇宙研究機構</org>で見ていました。\n<plc>インド</plc>は、<plc>アメリカ</plc>、昔の<plc>ソビエト</plc>、<plc>ヨーロッパ</plc>に続いて４番目に人工衛星を火星の軌道に入れました。<plc>アジア</plc>の国の中では、<plc>インド</plc>が初めてです。\n「マンガルヤーン」は、カメラやメタンガスを調べる機械を使って、軌道を回りながら火星を調べる予定です。使うお金は、全部で約４５億ルピー（<plc>日本</plc>のお金で約８０億円）です。<plc>アメリカ</plc>などに比べてとても安いため、<plc>インド</plc>はこれからもっと宇宙ビジネスをしていこうと考えています。',
 'article_easy_ruby': '<span class="colorL">インド</span>は<ruby>去年<rt>きょねん</rt></ruby>１１<ruby>月<rt>がつ</rt></ruby>、<ruby>火星<rt>かせい</rt></ruby>を<ruby>調<rt>しら</rt></ruby>べる<ruby>人工衛星<rt>じんこうえいせい</rt></ruby>「マンガルヤーン」を<ruby>打<rt>う</rt></ruby>ち<ruby>上<rt

In [40]:
js_e(dic)

In [30]:
urls[0]

'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/easy/k10014345501000/k10014345501000.html'

In [33]:
page = requests.get(urls[0]).text
snap = re.search(r'(between|1 time).*?<a href="(.+?)">', page)

In [34]:
snap

In [35]:
a = requests.get('https://web.archive.org/web/20150202040759/http://www3.nhk.or.jp/news/easy/k10014816211000/k10014816211000.html').text

In [36]:
scrape_easy_one(a)

{'id': 'k10014816211000',
 'title_easy': '地球温暖化が進むと日本はどうなるか\u3000環境省が考える',
 'title_easy_ruby': '<ruby>地球温暖化<rt>ちきゅうおんだんか</rt></ruby>が<ruby>進<rt>すす</rt></ruby>むと<ruby>日本<rt>にっぽん</rt></ruby>はどうなるか\u3000<ruby>環境省<rt>かんきょうしょう</rt></ruby>が<ruby>考<rt>かんが</rt></ruby>える',
 'article_easy': '<org>環境省</org>の審議会は、このまま地球温暖化が進むと<plc>日本</plc>はどうなるか考えました。\nそれによると、食料では、レベルがいちばん高い米の割合が減って、特に<plc>九州</plc>では２１世紀の終わりに４０％少なくなる可能性もあります。\n災害も多くなります。２１世紀の終わりには、川の水があふれるような激しい雨が増えたり、強い台風のため太平洋の海岸に高い波が来たりする危険が多くなります。\nそして、熱中症など暑さが原因で亡くなる可能性が、２０９０年から１０年の間に、最も多い場合、３．７倍になります。\n政府は、このような地球温暖化が原因で起きる被害を少なくするためにはどうしたらいいか、ことしの夏ごろまでに計画を作る予定です。',
 'article_easy_ruby': '<span class="colorC"><ruby>環境省<rt>かんきょうしょう</rt></ruby></span>の<ruby>審議会<rt>しんぎかい</rt></ruby>は、このまま<ruby>地球温暖化<rt>ちきゅうおんだんか</rt></ruby>が<ruby>進<rt>すす</rt></ruby>むと<span class="colorL"><ruby>日本<rt>にっぽん</rt></ruby></span>はどうなるか<ruby>考<rt>かんが</rt></ruby>えました。\nそれによると、<ruby>食料<rt>しょくりょう</rt></ruby>では、レベルがいちばん<ruby>高<rt>たか</rt></ruby>い<ruby>米<rt>こめ</rt></ruby>の<r